In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy
import networkx as nx
import geopandas as gpd
from IPython.display import clear_output
import seaborn as sns
import time
from random import shuffle
from collections import Counter, defaultdict
from utils import *
import random
from scipy.integrate import solve_ivp
import pandas as pd
from scipy.stats import poisson



%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams.update({'font.size': 10, 'font.style': 'normal'})

In [ ]:
status_color_map = {'S': '#5B958D', 'I': '#842D2D', 'R': '#ECAD74'}

# Preparing Data and Imports

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy
import networkx as nx
import geopandas as gpd
from IPython.display import clear_output
import seaborn as sns
import time
from random import shuffle
from collections import Counter, defaultdict
from utils import *
import random
from scipy.integrate import solve_ivp
import pandas as pd
from scipy.stats import poisson



%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams.update({'font.size': 10, 'font.style': 'normal'})

In [ ]:
status_color_map = {'S': '#5B958D', 'I': '#842D2D', 'R': '#ECAD74'}

In [ ]:
df = pd.read_csv('cleaned_housing_data.csv')
emplo = pd.read_csv('employment_by_ethnicity.csv')
inactivity = pd.read_csv('inactivity_in_london.csv')

In [ ]:
# Esegui outer join tra i tre DataFrame su ['area', 'year']
merged = pd.merge(df, emplo, on=['area', 'year'], how='outer')
merged = pd.merge(merged, inactivity, on=['area', 'year'], how='outer')

In [ ]:
cols_to_keep = ['date', 'area', 'no_of_crimes', 'year', 'median_salary','population_size', 'emp rate white UK born', 'emp rate white not UK born',
       'emp rate minority UK born', 'emp rate minority not UK born',
       'Economically Inactive']

merged_filtered = merged[cols_to_keep]
merged_filtered['inactivity_rate'] = (merged_filtered['Economically Inactive']/merged_filtered['population_size'])*100 
merged_filtered['crime_rate'] = (merged_filtered['no_of_crimes']/merged_filtered['population_size'])*1000
merged_filtered = merged_filtered[['date', 'area', 'year', 'crime_rate','median_salary', 'emp rate white UK born', 'emp rate white not UK born',
                                   'emp rate minority UK born', 'emp rate minority not UK born', 'inactivity_rate']]

In [ ]:
# Tenta la conversione in datetime per tutte le colonne
for col in merged_filtered.columns:
    if merged_filtered[col].dtype == 'object':
        try:
            merged_filtered[col] = pd.to_datetime(merged_filtered[col], errors='raise')
            print(f"✓ Converted '{col}' to datetime.")
        except:
            pass  

# Network construction: Socio-Economic Similarity in London

In [ ]:
adjacency_dict = {
    'barking and dagenham': ['newham', 'redbridge', 'havering', 'bexley', 'greenwich'],
    'barnet': ['harrow', 'brent', 'camden', 'haringey', 'enfield'],
    'bexley': ['greenwich', 'bromley', 'barking and dagenham', 'havering'],
    'brent': ['harrow', 'barnet', 'camden', 'westminster', 'kensington and chelsea', 'hammersmith and fulham', 'ealing'],
    'bromley': ['bexley', 'croydon', 'greenwich', 'lewisham', 'southwark'], 
    'camden': ['brent', 'barnet', 'haringey', 'islington', 'city of london', 'westminster'],
    'city of london': ['camden', 'islington', 'hackney', 'tower hamlets', 'westminster', 'southwark'],
    'croydon': ['bromley', 'lambeth', 'sutton', 'merton'],
    'ealing': ['brent', 'hammersmith and fulham', 'hounslow', 'hillingdon', 'harrow'],
    'enfield': ['barnet', 'haringey', 'waltham forest'],
    'greenwich': ['bexley', 'bromley' , 'lewisham', 'tower hamlets', 'newham', 'barking and dagenham'],
    'hackney': ['islington', 'haringey', 'waltham forest', 'tower hamlets', 'city of london','newham'],
    'hammersmith and fulham': ['kensington and chelsea', 'wandsworth', 'ealing', 'brent', 'richmond upon thames','hounslow'],
    'haringey': ['enfield', 'barnet', 'camden', 'islington', 'hackney', 'waltham forest'],
    'harrow': ['barnet', 'brent', 'hillingdon', 'ealing'],
    'havering': ['barking and dagenham', 'redbridge', 'bexley'],
    'hillingdon': ['harrow', 'ealing', 'hounslow'],
    'hounslow': ['hillingdon', 'ealing', 'richmond upon thames', 'hammersmith and fulham'],
    'islington': ['camden', 'hackney', 'haringey', 'city of london'],
    'kensington and chelsea': ['westminster', 'hammersmith and fulham', 'brent', 'wandsworth'],
    'kingston upon thames': ['richmond upon thames', 'merton', 'sutton', 'wandsworth'],
    'lambeth': ['wandsworth', 'southwark', 'croydon', 'merton', 'westminster'],
    'lewisham': ['greenwich', 'southwark', 'bromley','tower hamlets'],
    'merton': ['wandsworth', 'lambeth', 'croydon', 'kingston upon thames', 'sutton'],
    'newham': ['tower hamlets', 'hackney', 'waltham forest', 'redbridge', 'barking and dagenham', 'greenwich'],
    'redbridge': ['newham', 'barking and dagenham', 'havering', 'waltham forest'],
    'richmond upon thames': ['hounslow', 'wandsworth', 'kingston upon thames','hammersmith and fulham'],
    'southwark': ['lambeth', 'lewisham', 'tower hamlets', 'city of london'],
    'sutton': ['merton', 'croydon', 'kingston upon thames'],
    'tower hamlets': ['hackney', 'city of london', 'southwark', 'newham', 'greenwich','lewisham'],
    'waltham forest': ['enfield', 'haringey', 'hackney', 'newham', 'redbridge'],
    'wandsworth': ['richmond upon thames', 'hammersmith and fulham', 'kensington and chelsea', 'lambeth', 'merton','kingston upon thames','westminster'],
    'westminster': ['camden', 'city of london', 'kensington and chelsea', 'brent','wandsworth','lambeth']}

In [ ]:
network_base = merged_filtered.drop(columns=['date', 'crime_rate'])


def assign_period(year):
    if 2001 <= year <= 2006:
        return 1
    elif 2007 <= year <= 2012:
        return 2
    elif 2013 <= year <= 2018:
        return 3
    else:
        return None

network_base['period'] = network_base['year'].apply(assign_period)


network_df = (
    network_base
    .drop(columns='year')
    .groupby(['area', 'period'], as_index=False)
    .mean(numeric_only=True)
)

In [ ]:
network_df = network_df.sort_values(by=['period', 'area']).reset_index(drop=True)

In [ ]:
network_01_06 = network_df[network_df['period'] == 1]
network_07_12 = network_df[network_df['period'] == 2]
network_13_18 = network_df[network_df['period'] == 3]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Sotto-DataFrame per ciascun periodo
network_01_06 = network_df[network_df['period'] == 1]
network_07_12 = network_df[network_df['period'] == 2]
network_13_18 = network_df[network_df['period'] == 3]

# Funzione aggiornata
def describe_and_plot(df, label):
    df_clean = df.drop(columns=['area', 'period']).replace(0, np.nan)
    
    # Median salary separato
    if 'median_salary' in df_clean.columns:
        salary = df_clean['median_salary']
        df_other = df_clean.drop(columns='median_salary')
    else:
        salary = None
        df_other = df_clean

    # Statistiche variabili diverse da median_salary
    summary = df_other.describe().T[['min', '25%', '50%', '75%', 'max']]
    summary['mean'] = df_other.mean()
    summary = summary[['min', '25%', '50%', 'mean', '75%', 'max']]

    print(f"\n=== Summary statistics (excl. median_salary) for {label} ===")
    print(summary)

    # Boxplot
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=df_other)
    plt.xticks(rotation=45)
    plt.title(f"Boxplot of Variables (excl. median_salary) — {label}")
    plt.tight_layout()
    plt.show()

    # Statistiche e plot per median_salary
    if salary is not None:
        print(f"\n=== Median Salary Statistics for {label} ===")
        print(salary.describe())

        plt.figure(figsize=(6, 4))
        sns.boxplot(y=salary)
        plt.title(f"Boxplot of Median Salary — {label}")
        plt.tight_layout()
        plt.show()

# Applica la funzione a ciascun periodo
describe_and_plot(network_01_06, "2001–2006")
describe_and_plot(network_07_12, "2007–2012")
describe_and_plot(network_13_18, "2013–2018")


In [ ]:
def assign_levels(df):
    df = df.copy()

    # Inizializza le colonne
    df['white_UK'] = np.nan
    df['white_non_UK'] = np.nan

    # Condizione default: city of london → 3
    city_mask = df['area'] == 'city of london'
    df.loc[city_mask, 'white_UK'] = 3
    df.loc[city_mask, 'white_non_UK'] = 3

    # Per le altre righe — white_UK
    not_city = ~city_mask
    df.loc[not_city & (df['emp rate white UK born'] < 70), 'white_UK'] = 1
    df.loc[not_city & (df['emp rate white UK born'].between(70, 80, inclusive='both')), 'white_UK'] = 2
    df.loc[not_city & (df['emp rate white UK born'] > 80), 'white_UK'] = 3

    # Per le altre righe — white_non_UK
    df.loc[not_city & (df['emp rate white not UK born'] < 70), 'white_non_UK'] = 1
    df.loc[not_city & (df['emp rate white not UK born'].between(70, 80, inclusive='both')), 'white_non_UK'] = 2
    df.loc[not_city & (df['emp rate white not UK born'] > 80), 'white_non_UK'] = 3

    return df

# Applica la funzione a ciascun blocco
network_01_06 = assign_levels(network_01_06)
network_07_12 = assign_levels(network_07_12)
network_13_18 = assign_levels(network_13_18)

In [ ]:
def assign_minority_levels(df):
    df = df.copy()

    # Inizializza le colonne
    df['minority_UK'] = np.nan
    df['minority_non_UK'] = np.nan

    # Condizione default: city of london → 4
    city_mask = df['area'] == 'city of london'
    df.loc[city_mask, 'minority_UK'] = 4
    df.loc[city_mask, 'minority_non_UK'] = 4

    # Altre righe — minority_UK
    not_city = ~city_mask
    df.loc[not_city & (df['emp rate minority UK born'] < 50), 'minority_UK'] = 1
    df.loc[not_city & (df['emp rate minority UK born'].between(50, 60, inclusive='left')), 'minority_UK'] = 2
    df.loc[not_city & (df['emp rate minority UK born'].between(60, 70, inclusive='both')), 'minority_UK'] = 3
    df.loc[not_city & (df['emp rate minority UK born'] > 70), 'minority_UK'] = 4

    # Altre righe — minority_non_UK
    df.loc[not_city & (df['emp rate minority not UK born'] < 50), 'minority_non_UK'] = 1
    df.loc[not_city & (df['emp rate minority not UK born'].between(50, 60, inclusive='left')), 'minority_non_UK'] = 2
    df.loc[not_city & (df['emp rate minority not UK born'].between(60, 70, inclusive='both')), 'minority_non_UK'] = 3
    df.loc[not_city & (df['emp rate minority not UK born'] > 70), 'minority_non_UK'] = 4

    return df

# Applica la funzione a ciascun sotto-DataFrame
network_01_06 = assign_minority_levels(network_01_06)
network_07_12 = assign_minority_levels(network_07_12)
network_13_18 = assign_minority_levels(network_13_18)

In [ ]:
def assign_inactivity_group(df):
    df = df.copy()

    # Inizializza colonna
    df['inactivity_group'] = np.nan

    # City of London default = 2
    city_mask = df['area'] == 'city of london'
    df.loc[city_mask, 'inactivity_group'] = 2

    # Altri borough
    not_city = ~city_mask
    df.loc[not_city & (df['inactivity_rate'] < 22), 'inactivity_group'] = 1
    df.loc[not_city & (df['inactivity_rate'] >= 22), 'inactivity_group'] = 2

    return df

# Applica la funzione a ciascun sotto-DataFrame
network_01_06 = assign_inactivity_group(network_01_06)
network_07_12 = assign_inactivity_group(network_07_12)
network_13_18 = assign_inactivity_group(network_13_18)

In [ ]:
# Concatena i tre blocchi per operare insieme
salary_df = pd.concat([network_01_06, network_07_12, network_13_18], ignore_index=True)

# Raggruppa per area e periodo, calcola media salary (già fatto, ma lo rifacciamo per sicurezza)
avg_salary_by_period = (
    salary_df
    .groupby(['area', 'period'], as_index=False)['median_salary']
    .mean()
)

# Pivot per avere una colonna per ciascun periodo
salary_pivot = avg_salary_by_period.pivot(index='area', columns='period', values='median_salary')
salary_pivot.columns = ['avg_salary_01_06', 'avg_salary_07_12', 'avg_salary_13_18']

# Calcola i tassi di crescita
salary_pivot['growth_1_to_2'] = (salary_pivot['avg_salary_07_12'] - salary_pivot['avg_salary_01_06']) / salary_pivot['avg_salary_01_06']
salary_pivot['growth_2_to_3'] = (salary_pivot['avg_salary_13_18'] - salary_pivot['avg_salary_07_12']) / salary_pivot['avg_salary_07_12']

# Visualizza i risultati
print(salary_pivot[['growth_1_to_2', 'growth_2_to_3']].head())


In [ ]:
salary_pivot = salary_pivot[['growth_1_to_2','growth_2_to_3']]

In [ ]:
for column in salary_pivot.columns:
    salary_pivot[column] = salary_pivot[column]*100

In [ ]:
avg_growth_1_to_2 = salary_pivot['growth_1_to_2'].mean()

print(f'Average salary growth from period 1 to 2: {avg_growth_1_to_2:.4f}')

avg_growth_2_to_3 = salary_pivot['growth_2_to_3'].mean()

print(f'Average salary growth from period 2 to 3: {avg_growth_2_to_3:.4f}')

In [ ]:
# Step 1: Calcola soglie di crescita media
growth_1_to_2 = salary_pivot['growth_1_to_2'].mean()/100
growth_2_to_3 = salary_pivot['growth_2_to_3'].mean()/100

# Step 2: Definisci soglie salariali per ciascun periodo
threshold_1 = 25000
threshold_2 = threshold_1 * (1 + growth_1_to_2)
threshold_3 = threshold_2 * (1 + growth_2_to_3)

print(f"Thresholds: P1={threshold_1:.2f}, P2={threshold_2:.2f}, P3={threshold_3:.2f}")

# Step 3: Funzione per classificare un singolo dataset
def assign_salary_group(df, threshold):
    df = df.copy()

    # Calcola limiti per outlier (Q3 + 1.5 IQR)
    q1 = df['median_salary'].quantile(0.25)
    q3 = df['median_salary'].quantile(0.75)
    iqr = q3 - q1
    outlier_threshold = q3 + 1.5 * iqr

    # Inizializza la colonna
    df['salary_group'] = 2  # default

    # Assegna gruppo 3 per outlier
    df.loc[df['median_salary'] > outlier_threshold, 'salary_group'] = 3

    # Assegna gruppo 1 per valori sotto soglia, ma solo se non è già 3
    below_threshold = df['median_salary'] < threshold
    not_outlier = df['salary_group'] != 3
    df.loc[below_threshold & not_outlier, 'salary_group'] = 1

    return df

# Step 4: Applica alle tre finestre
network_01_06 = assign_salary_group(network_01_06, threshold_1)
network_07_12 = assign_salary_group(network_07_12, threshold_2)
network_13_18 = assign_salary_group(network_13_18, threshold_3)

In [ ]:
# Lista delle colonne da rimuovere
cols_to_drop = [
    'median_salary',
    'emp rate white UK born',
    'emp rate white not UK born',
    'emp rate minority UK born',
    'emp rate minority not UK born',
    'inactivity_rate'
]

# Elimina le colonne da ciascun DataFrame
network_01_06 = network_01_06.drop(columns=cols_to_drop)
network_07_12 = network_07_12.drop(columns=cols_to_drop)
network_13_18 = network_13_18.drop(columns=cols_to_drop)

# Verifica
print(network_01_06.head())


In [ ]:
# Step 1: Filtro solo anni > 2005
filtered = merged_filtered[merged_filtered['year'] > 2005]

# Step 2: Raggruppa per area e year, calcola media di tutte le colonne numeriche
annual_avg = (
    filtered
    .groupby(['area', 'year'], as_index=False)
    .mean(numeric_only=True)
)

In [ ]:
# Step 1: Rimuovi 'area' (non numerica)
corr_df = annual_avg.drop(columns=['area'])

# Step 2: Calcola la matrice di correlazione
corr_matrix = corr_df.corr()

# Step 3: Visualizza solo le correlazioni con 'crime_rate'
plt.figure(figsize=(10, 6))
sns.heatmap(
    corr_matrix[['crime_rate']].sort_values(by='crime_rate', ascending=False),
    annot=True, cmap='coolwarm', center=0
)
plt.title("Correlation of Variables with Crime Rate")
plt.tight_layout()
plt.show()

In [ ]:
weights = {
    'inactivity_group': 1,
    'white_non_UK': 1,
    'white_UK': 1,
    'minority_UK': 1,
    'salary_group': 1,
    'minority_non_UK': 1
}

In [ ]:
from itertools import product

# Funzione per calcolare la similarity matrix
def compute_similarity_matrix(df, label):
    df = df.copy()
    features = list(weights.keys())
    boroughs = df['area'].values
    matrix = pd.DataFrame(index=boroughs, columns=boroughs, dtype=float)

    # Normalizza pesi
    total_weight = sum(weights.values())

    # Calcolo pairwise
    for a, b in product(boroughs, repeat=2):
        row_a = df[df['area'] == a][features].iloc[0]
        row_b = df[df['area'] == b][features].iloc[0]

        score = sum(
            weights[f] for f in features if row_a[f] == row_b[f]
        ) / total_weight * 100  # percentuale
        matrix.loc[a, b] = round(score, 2)

    print(f"\n✅ Similarity matrix for {label} constructed.")
    return matrix

# Costruisci le 3 matrici per i tre periodi
sim_01_06 = compute_similarity_matrix(network_01_06, "2001–2006")
sim_07_12 = compute_similarity_matrix(network_07_12, "2007–2012")
sim_13_18 = compute_similarity_matrix(network_13_18, "2013–2018")

# (Opzionale) Visualizza un esempio
print(sim_13_18.head())


In [ ]:
def build_connections(sim_matrix, label):
    all_nodes = sim_matrix.index.tolist()
    connections = []

    for source in sim_matrix.index:
        for target in sim_matrix.columns:
            score = sim_matrix.loc[source, target]
            if source != target and score > 80:
                connections.append({'source': source, 'target': target})

    # Converti in DataFrame
    connections_df = pd.DataFrame(connections)

    # Crea DataFrame di nodi anche se isolati
    all_nodes_df = pd.DataFrame({'area': all_nodes})

    print(f"✅ Connections for {label} created with {len(connections_df)} edges and {len(all_nodes)} nodes.")
    return connections_df, all_nodes_df


# Applica la funzione ai tre periodi
connections_01_06, nodes_01_06 = build_connections(sim_01_06, "2001–2006")
connections_07_12, nodes_07_12 = build_connections(sim_07_12, "2007–2012")
connections_13_18, nodes_13_18 = build_connections(sim_13_18, "2013–2018")

In [ ]:
def add_geographic_edges(G, adjacency_dict):
    for source, neighbors in adjacency_dict.items():
        for target in neighbors:
            if G.has_edge(source, target) or source == target:
                continue
            if G.has_node(source) and G.has_node(target):
                G.add_edge(source, target, type='geo')
    return G

def build_combined_graph(connections_df, nodes_df, label):
    G = nx.Graph()

    # Aggiungi tutti i nodi, anche isolati
    G.add_nodes_from(nodes_df['area'])

    # Aggiungi connessioni da similarity > 80
    for _, row in connections_df.iterrows():
        G.add_edge(row['source'], row['target'], type='similarity')

    # Aggiungi connessioni geografiche
    G = add_geographic_edges(G, adjacency_dict)

    print(f"✅ Combined graph for {label} created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    return G

# Costruisci i grafi aggiornati
G_01_06 = build_combined_graph(connections_01_06, nodes_01_06, "2001–2006")
G_07_12 = build_combined_graph(connections_07_12, nodes_07_12, "2007–2012")
G_13_18 = build_combined_graph(connections_13_18, nodes_13_18, "2013–2018")

In [ ]:
def draw_colored_graph(G, title):
    pos = nx.spring_layout(G, seed=42)
    plt.figure(figsize=(10, 10))

    # Disegna nodi
    nx.draw_networkx_nodes(G, pos, node_size=500, node_color='lightblue')

    # Disegna archi di similarità
    sim_edges = [(u, v) for u, v, d in G.edges(data=True) if d['type'] == 'similarity']
    geo_edges = [(u, v) for u, v, d in G.edges(data=True) if d['type'] == 'geo']
    nx.draw_networkx_edges(G, pos, edgelist=sim_edges, width=2, edge_color='blue', label='similarity')
    nx.draw_networkx_edges(G, pos, edgelist=geo_edges, width=1.5, edge_color='green', style='dashed', label='geographic')

    # Etichette
    nx.draw_networkx_labels(G, pos, font_size=8)
    plt.title(title)
    plt.legend(loc='upper right')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualizza grafi combinati
draw_colored_graph(G_01_06, "Combined Network (2001–2006)")
draw_colored_graph(G_07_12, "Combined Network (2007–2012)")
draw_colored_graph(G_13_18, "Combined Network (2013–2018)")


# Network evolution setting different weights for socio-economic similarity computation

In [ ]:
from itertools import product

graphs_dict_01_06 = {}

# === Feature set ===
features = ['salary_group', 'inactivity_group', 'white_UK', 'white_non_UK', 'minority_UK', 'minority_non_UK']

# === Grid search weight configurations ===
weight_configs = []

# One-hot (1 feature at a time)
for f in features:
    weight_configs.append({'name': f'one_hot_{f}', 'weights': {k: (1 if k == f else 0) for k in features}})

# Equal weights
weight_configs.append({'name': 'equal', 'weights': {f: 1 for f in features}})

# Salary focus (3x)
w = {f: 1 for f in features}; w['salary_group'] = 3
weight_configs.append({'name': 'salary_focus', 'weights': w})

# Inactivity focus (3x)
w = {f: 1 for f in features}; w['inactivity_group'] = 3
weight_configs.append({'name': 'inactivity_focus', 'weights': w})

# White emp focus (2x)
w = {f: 1 for f in features}; w['white_UK'] = 2; w['white_non_UK'] = 2
weight_configs.append({'name': 'white_focus', 'weights': w})

# Minority emp focus (2x)
w = {f: 1 for f in features}; w['minority_UK'] = 2; w['minority_non_UK'] = 2
weight_configs.append({'name': 'minority_focus', 'weights': w})

# === Compute similarity matrix ===
def compute_similarity_matrix(df, weights):
    boroughs = df['area'].values
    matrix = pd.DataFrame(index=boroughs, columns=boroughs, dtype=float)
    total_weight = sum(weights.values())
    for a, b in product(boroughs, repeat=2):
        row_a = df[df['area'] == a][features].iloc[0]
        row_b = df[df['area'] == b][features].iloc[0]
        score = sum(weights[f] for f in features if row_a[f] == row_b[f])
        matrix.loc[a, b] = round(score / total_weight * 100, 2)
    return matrix

# === Crea grafo da similarity matrix (>80) ===
def build_graph(sim_matrix, adjacency_dict):
    G = nx.Graph()
    boroughs = sim_matrix.index.tolist()
    G.add_nodes_from(boroughs)

    # Add similarity edges
    for i in sim_matrix.index:
        for j in sim_matrix.columns:
            if i != j and sim_matrix.loc[i, j] > 80:
                G.add_edge(i, j, type='similarity')

    # Add geographic edges
    for source, neighbors in adjacency_dict.items():
        for target in neighbors:
            if source in boroughs and target in boroughs:
                if not G.has_edge(source, target):
                    G.add_edge(source, target, type='geo')
    return G

# === Riepilogo dei risultati ===
results = []

for config in weight_configs:
    name = config['name']
    weights = config['weights']
    sim_matrix = compute_similarity_matrix(network_01_06, weights)
    G = build_graph(sim_matrix, adjacency_dict)
    graphs_dict_01_06[name] = G

    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    num_components = nx.number_connected_components(G)
    largest_component_size = len(max(nx.connected_components(G), key=len))

    results.append({
        'config': name,
        'nodes': num_nodes,
        'edges': num_edges,
        'components': num_components,
        'largest_component_size': largest_component_size
    })



In [ ]:
def draw_colored_graph(G, title):
    pos = nx.spring_layout(G, seed=42)
    plt.figure(figsize=(10, 10))

    nx.draw_networkx_nodes(G, pos, node_size=500, node_color='lightblue')

    sim_edges = [(u, v) for u, v, d in G.edges(data=True) if d['type'] == 'similarity']
    geo_edges = [(u, v) for u, v, d in G.edges(data=True) if d['type'] == 'geo']

    nx.draw_networkx_edges(G, pos, edgelist=sim_edges, width=2, edge_color='blue', label='similarity')
    nx.draw_networkx_edges(G, pos, edgelist=geo_edges, width=1.5, edge_color='green', style='dashed', label='geographic')

    nx.draw_networkx_labels(G, pos, font_size=8)
    plt.title(title)
    plt.legend(loc='upper right')
    plt.axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
graphs_dict_01_06

In [ ]:
draw_colored_graph(graphs_dict_01_06['salary_focus'], "Graph: salary_focus")
draw_colored_graph(graphs_dict_01_06['one_hot_salary_group'], "Graph: one_hot_salary_group")
draw_colored_graph(graphs_dict_01_06['one_hot_inactivity_group'], "Graph: one_hot_inactivity_group")
draw_colored_graph(graphs_dict_01_06['one_hot_white_UK'], "Graph: one_hot_white_UK")
draw_colored_graph(graphs_dict_01_06['one_hot_white_non_UK'], "Graph: one_hot_white_non_UK")
draw_colored_graph(graphs_dict_01_06['one_hot_minority_UK'], "Graph: one_hot_minority_UK")
draw_colored_graph(graphs_dict_01_06['one_hot_minority_non_UK'], "Graph: one_hot_minority_non_UK")
draw_colored_graph(graphs_dict_01_06['equal'], "Graph: equal")
draw_colored_graph(graphs_dict_01_06['inactivity_focus'], "Graph: inactivity_focus")
draw_colored_graph(graphs_dict_01_06['white_focus'], "Graph: white_focus")
draw_colored_graph(graphs_dict_01_06['minority_focus'], "Graph: minority_focus")

In [ ]:
from itertools import product

graphs_dict_07_12 = {}

# === Feature set ===
features = ['salary_group', 'inactivity_group', 'white_UK', 'white_non_UK', 'minority_UK', 'minority_non_UK']

# === Grid search weight configurations ===
weight_configs = []

# One-hot (1 feature at a time)
for f in features:
    weight_configs.append({'name': f'one_hot_{f}', 'weights': {k: (1 if k == f else 0) for k in features}})

# Equal weights
weight_configs.append({'name': 'equal', 'weights': {f: 1 for f in features}})

# Salary focus (3x)
w = {f: 1 for f in features}; w['salary_group'] = 3
weight_configs.append({'name': 'salary_focus', 'weights': w})

# Inactivity focus (3x)
w = {f: 1 for f in features}; w['inactivity_group'] = 3
weight_configs.append({'name': 'inactivity_focus', 'weights': w})

# White emp focus (2x)
w = {f: 1 for f in features}; w['white_UK'] = 2; w['white_non_UK'] = 2
weight_configs.append({'name': 'white_focus', 'weights': w})

# Minority emp focus (2x)
w = {f: 1 for f in features}; w['minority_UK'] = 2; w['minority_non_UK'] = 2
weight_configs.append({'name': 'minority_focus', 'weights': w})

# === Compute similarity matrix ===
def compute_similarity_matrix(df, weights):
    boroughs = df['area'].values
    matrix = pd.DataFrame(index=boroughs, columns=boroughs, dtype=float)
    total_weight = sum(weights.values())
    for a, b in product(boroughs, repeat=2):
        row_a = df[df['area'] == a][features].iloc[0]
        row_b = df[df['area'] == b][features].iloc[0]
        score = sum(weights[f] for f in features if row_a[f] == row_b[f])
        matrix.loc[a, b] = round(score / total_weight * 100, 2)
    return matrix

# === Crea grafo da similarity matrix (>80) ===
def build_graph(sim_matrix, adjacency_dict):
    G = nx.Graph()
    boroughs = sim_matrix.index.tolist()
    G.add_nodes_from(boroughs)

    # Add similarity edges
    for i in sim_matrix.index:
        for j in sim_matrix.columns:
            if i != j and sim_matrix.loc[i, j] > 80:
                G.add_edge(i, j, type='similarity')

    # Add geographic edges
    for source, neighbors in adjacency_dict.items():
        for target in neighbors:
            if source in boroughs and target in boroughs:
                if not G.has_edge(source, target):
                    G.add_edge(source, target, type='geo')
    return G

# === Riepilogo dei risultati ===
results = []

for config in weight_configs:
    name = config['name']
    weights = config['weights']
    sim_matrix = compute_similarity_matrix(network_07_12, weights)
    G = build_graph(sim_matrix, adjacency_dict)
    graphs_dict_07_12[name] = G

    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    num_components = nx.number_connected_components(G)
    largest_component_size = len(max(nx.connected_components(G), key=len))

    results.append({
        'config': name,
        'nodes': num_nodes,
        'edges': num_edges,
        'components': num_components,
        'largest_component_size': largest_component_size
    })



In [ ]:
draw_colored_graph(graphs_dict_07_12['salary_focus'], "Graph: salary_focus")
draw_colored_graph(graphs_dict_07_12['one_hot_salary_group'], "Graph: one_hot_salary_group")
draw_colored_graph(graphs_dict_07_12['one_hot_inactivity_group'], "Graph: one_hot_inactivity_group")
draw_colored_graph(graphs_dict_07_12['one_hot_white_UK'], "Graph: one_hot_white_UK")
draw_colored_graph(graphs_dict_07_12['one_hot_white_non_UK'], "Graph: one_hot_white_non_UK")
draw_colored_graph(graphs_dict_07_12['one_hot_minority_UK'], "Graph: one_hot_minority_UK")
draw_colored_graph(graphs_dict_07_12['one_hot_minority_non_UK'], "Graph: one_hot_minority_non_UK")
draw_colored_graph(graphs_dict_07_12['equal'], "Graph: equal")
draw_colored_graph(graphs_dict_07_12['inactivity_focus'], "Graph: inactivity_focus")
draw_colored_graph(graphs_dict_07_12['white_focus'], "Graph: white_focus")
draw_colored_graph(graphs_dict_07_12['minority_focus'], "Graph: minority_focus")

In [ ]:
from itertools import product

graphs_dict_13_18 = {}

# === Feature set ===
features = ['salary_group', 'inactivity_group', 'white_UK', 'white_non_UK', 'minority_UK', 'minority_non_UK']

# === Grid search weight configurations ===
weight_configs = []

# One-hot (1 feature at a time)
for f in features:
    weight_configs.append({'name': f'one_hot_{f}', 'weights': {k: (1 if k == f else 0) for k in features}})

# Equal weights
weight_configs.append({'name': 'equal', 'weights': {f: 1 for f in features}})

# Salary focus (3x)
w = {f: 1 for f in features}; w['salary_group'] = 3
weight_configs.append({'name': 'salary_focus', 'weights': w})

# Inactivity focus (3x)
w = {f: 1 for f in features}; w['inactivity_group'] = 3
weight_configs.append({'name': 'inactivity_focus', 'weights': w})

# White emp focus (2x)
w = {f: 1 for f in features}; w['white_UK'] = 2; w['white_non_UK'] = 2
weight_configs.append({'name': 'white_focus', 'weights': w})

# Minority emp focus (2x)
w = {f: 1 for f in features}; w['minority_UK'] = 2; w['minority_non_UK'] = 2
weight_configs.append({'name': 'minority_focus', 'weights': w})

# === Compute similarity matrix ===
def compute_similarity_matrix(df, weights):
    boroughs = df['area'].values
    matrix = pd.DataFrame(index=boroughs, columns=boroughs, dtype=float)
    total_weight = sum(weights.values())
    for a, b in product(boroughs, repeat=2):
        row_a = df[df['area'] == a][features].iloc[0]
        row_b = df[df['area'] == b][features].iloc[0]
        score = sum(weights[f] for f in features if row_a[f] == row_b[f])
        matrix.loc[a, b] = round(score / total_weight * 100, 2)
    return matrix

# === Crea grafo da similarity matrix (>80) ===
def build_graph(sim_matrix, adjacency_dict):
    G = nx.Graph()
    boroughs = sim_matrix.index.tolist()
    G.add_nodes_from(boroughs)

    # Add similarity edges
    for i in sim_matrix.index:
        for j in sim_matrix.columns:
            if i != j and sim_matrix.loc[i, j] > 80:
                G.add_edge(i, j, type='similarity')

    # Add geographic edges
    for source, neighbors in adjacency_dict.items():
        for target in neighbors:
            if source in boroughs and target in boroughs:
                if not G.has_edge(source, target):
                    G.add_edge(source, target, type='geo')
    return G

# === Riepilogo dei risultati ===
results = []

for config in weight_configs:
    name = config['name']
    weights = config['weights']
    sim_matrix = compute_similarity_matrix(network_13_18, weights)
    G = build_graph(sim_matrix, adjacency_dict)
    graphs_dict_13_18[name] = G

    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    num_components = nx.number_connected_components(G)
    largest_component_size = len(max(nx.connected_components(G), key=len))

    results.append({
        'config': name,
        'nodes': num_nodes,
        'edges': num_edges,
        'components': num_components,
        'largest_component_size': largest_component_size
    })



In [ ]:
draw_colored_graph(graphs_dict_13_18['salary_focus'], "Graph: salary_focus")
draw_colored_graph(graphs_dict_13_18['one_hot_salary_group'], "Graph: one_hot_salary_group")
draw_colored_graph(graphs_dict_13_18['one_hot_inactivity_group'], "Graph: one_hot_inactivity_group")
draw_colored_graph(graphs_dict_13_18['one_hot_white_UK'], "Graph: one_hot_white_UK")
draw_colored_graph(graphs_dict_13_18['one_hot_white_non_UK'], "Graph: one_hot_white_non_UK")
draw_colored_graph(graphs_dict_13_18['one_hot_minority_UK'], "Graph: one_hot_minority_UK")
draw_colored_graph(graphs_dict_13_18['one_hot_minority_non_UK'], "Graph: one_hot_minority_non_UK")
draw_colored_graph(graphs_dict_13_18['equal'], "Graph: equal")
draw_colored_graph(graphs_dict_13_18['inactivity_focus'], "Graph: inactivity_focus")
draw_colored_graph(graphs_dict_13_18['white_focus'], "Graph: white_focus")
draw_colored_graph(graphs_dict_13_18['minority_focus'], "Graph: minority_focus")

# Crime Rate analysis

In [ ]:
cols_to_keep = ['date','area','year','crime_rate']
merged_filtered = merged_filtered[cols_to_keep]

In [ ]:
# Step 1: Assicurati che 'date' sia datetime
merged_filtered['date'] = pd.to_datetime(merged_filtered['date'], errors='coerce')

# Step 2: Crea colonna 'semester': 1 se mese <=6, 2 altrimenti
merged_filtered['semester'] = merged_filtered['date'].dt.month.apply(lambda m: 1 if m <= 6 else 2)

# Step 3: Groupby per area, anno e semestre — calcola media crime_rate
crime_semester_avg = (
    merged_filtered
    .groupby(['area', 'year', 'semester'], as_index=False)['crime_rate']
    .mean()
)

# Step 4: Crea colonna temporale continua per asse X: es. 2017.0, 2017.5
crime_semester_avg['time'] = crime_semester_avg['year'] + (crime_semester_avg['semester'] - 1) * 0.5

# Step 5: Plot dell’evoluzione del crime rate per borough
plt.figure(figsize=(14, 8))

for area in crime_semester_avg['area'].unique():
    area_data = crime_semester_avg[crime_semester_avg['area'] == area]
    plt.plot(area_data['time'], area_data['crime_rate'], label=area, alpha=0.6)

plt.title("Crime Rate Evolution per Borough (Semiannual Aggregation)")
plt.xlabel("Time (Year + 0.5)")
plt.ylabel("Crime Rate")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Calcola le statistiche di sintesi per ciascun borough
crime_summary = (
    crime_semester_avg
    .groupby('area')['crime_rate']
    .agg([
        ('min', 'min'),
        ('25%', lambda x: x.quantile(0.25)),
        ('50%', lambda x: x.quantile(0.50)),
        ('70%', lambda x: x.quantile(0.70)),
        ('max', 'max'),
        ('avg', 'mean')
    ])
    .reset_index()
)

# Visualizza le prime righe
print(crime_summary.head())


In [ ]:
# Calcola media e mediana
mean_crime = crime_semester_avg['crime_rate'].mean()
median_crime = crime_semester_avg['crime_rate'].median()

# Stampa i risultati
print(f"Valore medio di crime_rate: {mean_crime:.2f}")
print(f"Valore mediano di crime_rate: {median_crime:.2f}")

In [ ]:
# Definisci soglia
crime_threshold = 7.94

# Aggiungi colonna 'status'
crime_semester_avg['status'] = crime_semester_avg['crime_rate'].apply(
    lambda x: 'I' if x < crime_threshold else 'S'
)

In [ ]:
# Groupby e conteggio dei quartieri per status in ogni periodo
status_counts = (
    crime_semester_avg
    .groupby(['year', 'semester', 'status'])
    .size()
    .reset_index(name='count')
)

# Pivot per avere colonne separate per 'I' e 'S'
status_summary = status_counts.pivot(index=['year', 'semester'], columns='status', values='count').fillna(0)

# Rinomina colonne per chiarezza (opzionale)
status_summary.columns.name = None
status_summary = status_summary.rename(columns={'I': 'safe_count', 'S': 'dangerous_count'}).astype(int)

In [ ]:
# Aggiungi colonna temporale continua: 2001.0 per semestre 1, 2001.5 per semestre 2, ecc.
status_summary = status_summary.reset_index()
status_summary['time'] = status_summary['year'] + (status_summary['semester'] - 1) * 0.5

# Riordina le colonne
status_summary = status_summary[['time', 'year', 'semester', 'safe_count', 'dangerous_count']]

In [ ]:
# Media tra i due semestri per ciascun anno
annual_status = (
    status_summary
    .groupby('year')[['safe_count', 'dangerous_count']]
    .mean()
    .reset_index()
)

In [ ]:
# Calcola rolling average su 3 anni
rolling_3yr = (
    annual_status
    .set_index('year')
    .rolling(window=3, min_periods=1)
    .mean()
    .reset_index()
)

# Plot
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

plt.plot(rolling_3yr['year'], rolling_3yr['safe_count'], label='Safe (3yr avg)', color='blue', marker='o')
plt.plot(rolling_3yr['year'], rolling_3yr['dangerous_count'], label='Dangerous (3yr avg)', color='red', marker='o')

plt.title("Evoluzione del numero medio di quartieri Safe vs Dangerous (media 3 anni)")
plt.xlabel("Anno")
plt.ylabel("Media 3 anni - numero di quartieri")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Calcola media mobile triennale
rolling_3yr = (
    annual_status
    .set_index('year')
    .rolling(window=3, min_periods=1)
    .mean()
    .reset_index()
)

# Filtra solo anni tra 2004 e 2015
rolling_filtered = rolling_3yr[(rolling_3yr['year'] >= 2004) & (rolling_3yr['year'] <= 2015)]

# Plot
plt.figure(figsize=(12, 6))

plt.plot(rolling_filtered['year'], rolling_filtered['safe_count'], label='Safe (3yr avg)', color='blue', marker='o')
plt.plot(rolling_filtered['year'], rolling_filtered['dangerous_count'], label='Dangerous (3yr avg)', color='red', marker='o')

plt.title("SI spreading pattern")
plt.xlabel("Anno")
plt.ylabel("Media 3 anni - numero di quartieri")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# Network based SI

In [ ]:
class SIModel_nx:
    def __init__(self, 
                 G,
                 infection_rate, 
                 statuses_distribution=[0.8,0.2]):
        self.G = G
        self.infection_rate = infection_rate
        self.history = []
        
        self.setup_agents(statuses_distribution)

    def setup_agents(self, statuses_distribution):
        statuses = ['S', 'I']
        num_nodes = self.G.number_of_nodes()
        
        # Inizializza gli stati dei nodi
        nodes_status = {n: s for n, s in zip(list(self.G.nodes),
                                             random.choices(statuses, weights=statuses_distribution, k=num_nodes))}        
        
        nx.set_node_attributes(self.G, nodes_status, 'status')

    def run_step(self):
        nodes_list = list(self.G.nodes)
        random.shuffle(nodes_list)
        
        # Copia degli stati attuali per aggiornamenti simultanei
        new_statuses = nx.get_node_attributes(self.G, 'status')
        
        for n in nodes_list:
            if self.G.nodes[n]['status'] == 'I':
                # Infetta i vicini
                neighbors = list(self.G.neighbors(n))
                for neighbor in neighbors:
                    if self.G.nodes[neighbor]['status'] == 'S':
                        # Applica l'infezione con probabilità modificata
                        if random.random() < self.infection_rate:
                            new_statuses[neighbor] = 'I'
        
        # Aggiorna gli stati dei nodi
        nx.set_node_attributes(self.G, new_statuses, 'status')
        
        # Registra lo stato della popolazione
        self.history.append(self.get_population_status())

    def get_population_status(self):
        status_counts = {'S': 0, 'I': 0}
        counts = Counter(nx.get_node_attributes(self.G, 'status').values())
        status_counts.update(dict(counts))
        
        return status_counts

    def reset(self, statuses_distribution=[0.8,0.2]):
        # Resetta gli stati dei nodi
        self.setup_agents(statuses_distribution)
        self.history = []

In [ ]:
simulation_runs = 200
time_steps = 22  # 4 + 12 + 6

S_all = pd.DataFrame()
I_all = pd.DataFrame()

for i in range(simulation_runs):
    # Inizializza primo grafo (2001–2006)
    G1 = graphs_dict_01_06['equal'].copy()
    model = SIModel_nx(G=G1, infection_rate=0.01, statuses_distribution=[0.80, 0.20])

    history = []

    for t in range(time_steps):
        if t == 4:
            # Passaggio a grafo 2007–2012
            G2 = graphs_dict_07_12['equal'].copy()

            # Copia status corrente dal vecchio grafo
            current_status = nx.get_node_attributes(model.G, 'status')
            nx.set_node_attributes(G2, current_status, 'status')

            # Aggiorna grafo nel modello
            model.G = G2

        elif t == 16:
            # Passaggio a grafo 2013–2018
            G3 = graphs_dict_13_18['equal'].copy()

            # Copia status corrente dal grafo precedente
            current_status = nx.get_node_attributes(model.G, 'status')
            nx.set_node_attributes(G3, current_status, 'status')

            # Aggiorna grafo nel modello
            model.G = G3

        model.run_step()
        history.append(model.history[-1])  # registra stato

    # Trasforma la run in DataFrame
    RESt = pd.DataFrame.from_records(history)
    S_all = pd.concat([S_all, RESt['S'].to_frame()], axis=1)
    I_all = pd.concat([I_all, RESt['I'].to_frame()], axis=1)



In [ ]:
# PLOT
fig, axs = plt.subplots(nrows=1, ncols=2, facecolor='w', figsize=(10,3))
t = np.arange(0,time_steps,1)

ax = axs[0]
ax.plot(t, S_all.mean(1), opinion_colors[1], alpha=0.95, lw=2, label= 'Dangerous')
ax.plot(t, I_all.mean(1), opinion_colors[-1], alpha=0.95, lw=2, label= 'Safe')
ax.fill_between(t, S_all.mean(1) - 1*S_all.std(1), S_all.mean(1) + 1*S_all.std(1), color=opinion_colors[1], alpha=0.1)
ax.fill_between(t, I_all.mean(1) - 1*I_all.std(1), I_all.mean(1) + 1*I_all.std(1), color=opinion_colors[-1], alpha=0.1)
ax.set_ylim(0, len(G3))
set_plot_style(ax, title='SI dynamics', xlabel='time', ylabel='Neighbours')

ax = axs[1]
ax.scatter(S_all, I_all ,s=5, facecolor='k', alpha=0.1)
ax.scatter(S_all.iloc[0], I_all.iloc[0], s=45, facecolor=opinion_colors[-1], alpha=.2, label='start', marker='^')
ax.scatter(S_all.iloc[-1], I_all.iloc[-1], s=45, facecolor=opinion_colors[1], alpha=.2, label='end', marker='v')

set_plot_style(ax, title='Phase graph', xlabel='S', ylabel='I')

# Fine Tuning of Beta and Network Structure to fit the data

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

# Parametri
beta_values = np.linspace(0.0, 0.1, 20)
simulation_runs = 200
time_steps = 23 

# Target: DANGEROUS COUNT da rolling_filtered
target_years = rolling_filtered['year'].values
target_values = rolling_filtered['dangerous_count'].values

mle_results = []

for net_name, G_base in graphs_dict_01_06.items():
    print(f"🔄 Network: {net_name}")

    for beta in beta_values:
        dang_matrix = []

        for _ in range(simulation_runs):
            G1 = graphs_dict_01_06[net_name].copy()
            model = SIModel_nx(G=G1, infection_rate=beta, statuses_distribution=[0.80, 0.20])
            history = []

            for t in range(time_steps):
                if t == 4:
                    G2 = graphs_dict_07_12[net_name].copy()
                    current_status = nx.get_node_attributes(model.G, 'status')
                    nx.set_node_attributes(G2, current_status, 'status')
                    model.G = G2
                elif t == 16:
                    G3 = graphs_dict_13_18[net_name].copy()
                    current_status = nx.get_node_attributes(model.G, 'status')
                    nx.set_node_attributes(G3, current_status, 'status')
                    model.G = G3

                model.run_step()
                history.append(model.history[-1])

            S_series = [step['S'] for step in history]
            dang_matrix.append(S_series)

        dang_avg = np.mean(dang_matrix, axis=0)

        sim_dict = {2004 + t // 2: dang_avg[t] for t in range(0, time_steps, 2) if t < len(dang_avg)}

        filtered_years = [y for y in target_years if y in sim_dict]
        filtered_targets = [target_values[list(target_years).index(y)] for y in filtered_years]
        pred = [sim_dict[y] for y in filtered_years]

        residuals = np.array(filtered_targets) - np.array(pred)
        n_obs = len(filtered_targets)

        if n_obs > 0:
            sigma2_mle = np.mean(residuals ** 2)
            llh = -0.5 * n_obs * np.log(2 * np.pi * sigma2_mle) - np.sum(residuals ** 2) / (2 * sigma2_mle)
        else:
            sigma2_mle = np.nan
            llh = -np.inf

        mle_results.append({
            'network': net_name,
            'beta': beta,
            'sigma2': sigma2_mle,
            'log_likelihood': llh
        })

# Riepilogo
mle_df = pd.DataFrame(mle_results)
best_mle = mle_df.sort_values(by='log_likelihood', ascending=False).head(10)
print(best_mle)


In [ ]:
avg_llh_per_network = mle_df.groupby('network')['log_likelihood'].mean().reset_index()
avg_llh_per_network = avg_llh_per_network.sort_values(by='log_likelihood', ascending=False).reset_index(drop = True)
avg_llh_per_network

In [ ]:
avg_llh_per_beta = mle_df.groupby('beta')['log_likelihood'].mean().reset_index()
avg_llh_per_beta = avg_llh_per_beta.sort_values(by='log_likelihood', ascending=False).reset_index(drop = True)
avg_llh_per_beta

In [ ]:
simulation_runs = 200
time_steps = 23  

S_all = pd.DataFrame()
I_all = pd.DataFrame()

for i in range(simulation_runs):
    G1 = graphs_dict_01_06['minority_focus'].copy()
    model = SIModel_nx(G=G1, infection_rate=0.016, statuses_distribution=[0.80, 0.20])

    history = []

    for t in range(time_steps):
        if t == 4:
            G2 = graphs_dict_07_12['minority_focus'].copy()

            # Copia status corrente dal vecchio grafo
            current_status = nx.get_node_attributes(model.G, 'status')
            nx.set_node_attributes(G2, current_status, 'status')

            # Aggiorna grafo nel modello
            model.G = G2

        elif t == 16:
            # Passaggio a grafo 2013–2018
            G3 = graphs_dict_13_18['minority_focus'].copy()

            # Copia status corrente dal grafo precedente
            current_status = nx.get_node_attributes(model.G, 'status')
            nx.set_node_attributes(G3, current_status, 'status')

            # Aggiorna grafo nel modello
            model.G = G3

        model.run_step()
        history.append(model.history[-1])  # registra stato

    # Trasforma la run in DataFrame
    RESt = pd.DataFrame.from_records(history)
    S_all = pd.concat([S_all, RESt['S'].to_frame()], axis=1)
    I_all = pd.concat([I_all, RESt['I'].to_frame()], axis=1)



In [ ]:
# PLOT
fig, axs = plt.subplots(nrows=1, ncols=2, facecolor='w', figsize=(10,3))
t = np.arange(0,time_steps,1)

ax = axs[0]
ax.plot(t, S_all.mean(1), opinion_colors[1], alpha=0.95, lw=2, label= 'Dangerous')
ax.plot(t, I_all.mean(1), opinion_colors[-1], alpha=0.95, lw=2, label= 'Safe')
ax.fill_between(t, S_all.mean(1) - 1*S_all.std(1), S_all.mean(1) + 1*S_all.std(1), color=opinion_colors[1], alpha=0.1)
ax.fill_between(t, I_all.mean(1) - 1*I_all.std(1), I_all.mean(1) + 1*I_all.std(1), color=opinion_colors[-1], alpha=0.1)
ax.set_ylim(0, len(G3))
set_plot_style(ax, title='SI dynamics', xlabel='time', ylabel='Neighbours')

ax = axs[1]
ax.scatter(S_all, I_all ,s=5, facecolor='k', alpha=0.1)
ax.scatter(S_all.iloc[0], I_all.iloc[0], s=45, facecolor=opinion_colors[-1], alpha=.2, label='start', marker='^')
ax.scatter(S_all.iloc[-1], I_all.iloc[-1], s=45, facecolor=opinion_colors[1], alpha=.2, label='end', marker='v')

set_plot_style(ax, title='Phase graph', xlabel='S', ylabel='I')

# Evolution in 40 years from 2005 according to our model

In [ ]:
simulation_runs = 200
time_steps = 53  # 4 + 12 + 6

S_all = pd.DataFrame()
I_all = pd.DataFrame()

for i in range(simulation_runs):
    G1 = graphs_dict_01_06['minority_focus'].copy()
    model = SIModel_nx(G=G1, infection_rate=0.016, statuses_distribution=[0.80, 0.20])

    history = []

    for t in range(time_steps):
        if t == 4:
            G2 = graphs_dict_07_12['minority_focus'].copy()

            # Copia status corrente dal vecchio grafo
            current_status = nx.get_node_attributes(model.G, 'status')
            nx.set_node_attributes(G2, current_status, 'status')

            # Aggiorna grafo nel modello
            model.G = G2

        elif t == 16:
            # Passaggio a grafo 2013–2018
            G3 = graphs_dict_13_18['minority_focus'].copy()

            # Copia status corrente dal grafo precedente
            current_status = nx.get_node_attributes(model.G, 'status')
            nx.set_node_attributes(G3, current_status, 'status')

            # Aggiorna grafo nel modello
            model.G = G3

        model.run_step()
        history.append(model.history[-1])  # registra stato

    # Trasforma la run in DataFrame
    RESt = pd.DataFrame.from_records(history)
    S_all = pd.concat([S_all, RESt['S'].to_frame()], axis=1)
    I_all = pd.concat([I_all, RESt['I'].to_frame()], axis=1)



In [ ]:
# PLOT
fig, axs = plt.subplots(nrows=1, ncols=2, facecolor='w', figsize=(10,3))
t = np.arange(0,time_steps,1)

ax = axs[0]
ax.plot(t, S_all.mean(1), opinion_colors[1], alpha=0.95, lw=2, label= 'Dangerous')
ax.plot(t, I_all.mean(1), opinion_colors[-1], alpha=0.95, lw=2, label= 'Safe')
ax.fill_between(t, S_all.mean(1) - 1*S_all.std(1), S_all.mean(1) + 1*S_all.std(1), color=opinion_colors[1], alpha=0.1)
ax.fill_between(t, I_all.mean(1) - 1*I_all.std(1), I_all.mean(1) + 1*I_all.std(1), color=opinion_colors[-1], alpha=0.1)
ax.set_ylim(0, len(G3))
set_plot_style(ax, title='SI dynamics', xlabel='time', ylabel='Neighbours')

ax = axs[1]
ax.scatter(S_all, I_all ,s=5, facecolor='k', alpha=0.1)
ax.scatter(S_all.iloc[0], I_all.iloc[0], s=45, facecolor=opinion_colors[-1], alpha=.2, label='start', marker='^')
ax.scatter(S_all.iloc[-1], I_all.iloc[-1], s=45, facecolor=opinion_colors[1], alpha=.2, label='end', marker='v')

set_plot_style(ax, title='Phase graph', xlabel='S', ylabel='I')